### Indroduction
#### This notebook consists code for the Neural Style Transfer using pre-trained CNN classifier on Video data to produce stylized video 

In [0]:
!pip install hyperas

In [18]:
from google.colab import files
files.upload()

Saving input_video.avi to input_video.avi


{'input_video.avi': b'RIFF\x94\x01\x0e\x00AVI LIST\xdc\x00\x00\x00hdrlavih8\x00\x00\x00P\xc3\x00\x00\xff\xe0\xf5\x05\x00\x00\x00\x00\x10\t\x00\x00\x13\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x10\x00\x80\x02\x00\x00\xe0\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00LISTt\x00\x00\x00strlstrh8\x00\x00\x00vidsMJPG\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00\x13\x00\x00\x00\x00\x00\x10\x00\xff\xff\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00\x80\x02\xe0\x01strf(\x00\x00\x00(\x00\x00\x00\x80\x02\x00\x00\xe0\x01\x00\x00\x01\x00\x18\x00MJPG\x00\x10\x0e\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00LIST\x14\x00\x00\x00odmldmlh\x08\x00\x00\x00\x13\x00\x00\x00\x00\x00\x00\x00JUNK\x08\x0f\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [0]:
#Necessary Packages
from __future__ import print_function, division
#from __future__ import print_function, division
from builtins import range, input
from keras.preprocessing import image   # for preprocessing the images
from builtins import range,input
from datetime import datetime
import scipy
import cv2    # for capturing videos
import math    # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils, to_categorical
from skimage.transform import resize   # for resizing images
import os
import tqdm
from tqdm import tqdm,tqdm_pandas
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras import applications,models, losses,optimizers
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_last')
import cv2
from glob import glob
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
import tqdm
import cv2
from keras import applications
from keras.models import Model
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Convolution2D, MaxPooling2D,LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import Input, Dense, Flatten, Dropout, Activation, Lambda, Permute, Reshape
import numpy as np
import keras.backend as K
import scipy as sp
from scipy.spatial import distance
from PIL import Image
from keras.preprocessing import image
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from hyperas.distributions import choice, uniform #conditional
import hyperopt
from sklearn.model_selection import train_test_split
from keras.utils import Sequence, to_categorical
from sklearn.model_selection import StratifiedKFold
import sklearn.metrics
from keras.callbacks import (EarlyStopping, LearningRateScheduler,
                             ModelCheckpoint, TensorBoard, ReduceLROnPlateau)
from keras.layers import (Convolution1D, Dense, Dropout, GlobalAveragePooling1D, 
                          GlobalMaxPool1D, Input, MaxPool1D, concatenate)
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from os import listdir
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding,LSTM,LSTMCell
from keras.layers.merge import add
from keras.models import Model
from keras.utils import plot_model
from nltk.translate.bleu_score import corpus_bleu
from keras.models import load_model
from skimage.io import imread, imshow, imread_collection, concatenate_images
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

### Helper function for preprocessing content and style image before feed to VGG-16. We need to make sure the shape will be  (224,224),as per VGG-16 standard input format.

In [7]:
def load_preprocess_img(p,shape = None):
    Img = image.load_img(p, target_size=shape)
    X = image.img_to_array(Img)
    X = np.expand_dims(X,axis=0)    
    X = preprocess_input(X)
    return X

def preprocess_img(frame,shape = None):
    X = np.expand_dims(frame,axis=0)    
    X = preprocess_input(X.astype(('float64')))
    return X
    

#Loading style image
    
style_img = load_preprocess_img(p = "Style_Image.jpg", shape=(224,224))
batch_shape = style_img.shape
shape = style_img.shape[1:]
shape

(224, 224, 3)

### Some helper functions:

#### a) vgg_avg_pooling customizes keras standard VGG16. Here we want to account for features across the entire image so get rid of the maxpool which throws away information

#### b) vgg_cutoff freezes the output(activation map)layer of earlier defined vgg_avg_pool model.there are 13 convolutions in total we can pick any of deeper layer as the "output" of our content model

#### c) gram_matrix defines calculation of Gram matrix for network feature maps

#### d) After gram_matrix calculation we will use style_loss function to total style loss of the style network.

#### e) unpreprocess and f) scale(optional) are to restore/modify the image pixel values generated from VGG-16

In [0]:
shape = (224,224,3)

#Content model define
def vgg_avg_pooling(shape):
    vgg = VGG16(input_shape=shape,weights='imagenet',include_top=False)
    model = Sequential()
    for layer in vgg.layers:
        if layer.__class__ == MaxPooling2D:
        # replace it with average pooling    
            model.add(AveragePooling2D())
        else:
            model.add(layer)
    return model   

def vgg_cutoff(shape,num_conv):
    if num_conv<1|num_conv>13:
        print('Error layer must be with in [1,13]')
    model = vgg_avg_pooling(shape)
    new_model = Sequential()
    n=0
    for layer in model.layers:
        new_model.add(layer)
        if layer.__class__ == Conv2D:
            n+=1
        if n >= num_conv:
            break
    return new_model

#Style loss comutation graph

def gram_matrix(img):
    # input is (H, W, C) (C = # feature maps)
    # we first need to convert it to (C, H*W)
    X = K.batch_flatten(K.permute_dimensions(img,(2,0,1)))
    # now, calculate the gram matrix
    # gram = XX^T / N
    gram_mat = K.dot(X,K.transpose(X))/img.get_shape().num_elements()
    return gram_mat 

def style_loss(y,t):
    return K.mean(K.square(gram_matrix(y)-gram_matrix(t)))

def unpreprocess(img):
    img[..., 0] += 103.939
    img[..., 1] += 116.779
    img[..., 2] += 126.68
    img = img[..., ::-1]
    return img

def scale(x):
    x = x-x.min()
    x=x/x.max()
    return x

#### This code block describes network and necessary computation for style image.We don't apply vgg_cutoff function here as the idea is to form a multi-output model combining shallow and deep conv layers to calculate style loss for style representation.

In [0]:
#loading,processing and defining multi_output_model and style loss computation of style image

#Style image load and VGG model load.
path = "Style_Image.jpg"
img = image.load_img(path)
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)    
x = preprocess_input(x)
    #shape
batch_shape = x.shape
shape = x.shape[1:]
vgg = vgg_avg_pooling(shape)

#Define multi-output model
symb_conv_outputs = [layer.get_output_at(1) for layer in vgg.layers if layer.name.endswith('conv1')]
multi_output_model = Model(vgg.input, symb_conv_outputs)
symb_layer_out = [K.variable(y) for y in multi_output_model.predict(x)]

#Conv layer weight matrix
weights = [0.2,0.4,0.3,0.5,0.2]    
loss=0
#Total style loss
for symb,actual,w in zip(symb_conv_outputs,symb_layer_out,weights):
    loss += w * style_loss(symb[0],actual[0])
    
#gradients which are needed by the optimizer    
grad = K.gradients(loss,multi_output_model.input)
get_loss_grad = K.function(inputs=[multi_output_model.input], outputs=[loss] + grad)

#Scipy's minimizer function(fmin_l_bfgs_b) allows us to pass back function value f(x) and 
#its gradient f'(x), which we calculated in earlier step. 
#However, we need to unroll the input to minimizer function in1-D array format and both loss and gradient must be np.float64.

def get_loss_grad_wrapper(x_vec):
    l,g = get_loss_grad([x_vec.reshape(*batch_shape)])
    return l.astype(np.float64), g.flatten().astype(np.float64)    

In [0]:
import keras
keras.__version__
from keras.backend import function

### L-BFGS optimizer to minimize overall network loss. Here we directly feed our wrapper function to calculate loss and gradient in each iteration. 

In [0]:
#Function to minimize loss
def min_loss(fn,epochs,batch_shape):
    t0 = datetime.now()
    losses = []
    x = np.random.randn(np.prod(batch_shape))
    for i in range(epochs):
        x, l, _ = scipy.optimize.fmin_l_bfgs_b(func=fn,x0=x,maxfun=20)
    # bounds=[[-127, 127]]*len(x.flatten())
    #x = np.clip(x, -127, 127)
    # print("min:", x.min(), "max:", x.max())
        print("iter=%s, loss=%s" % (i, l))
        losses.append(l)
    print("duration:", datetime.now() - t0)
    plt.plot(losses)
    plt.show()

    newimg = x.reshape(*batch_shape)
    final_img = unpreprocess(newimg)
    return final_img[0]

### Put together every thing in a for loop extracting image frame and iterating individual image frame through the NST network and saving stylized images.  

In [0]:
import cv2
import numpy as np
 
# Create a VideoCapture object
batch_shape = style_img.shape
shape = style_img.shape[1:]
cap = cv2.VideoCapture("input_video.avi")
frame_width = int(cap.get(3))
frame_height = int(cap.get(4)) 
# Check if camera opened successfully
if (cap.isOpened() == False): 
    print("Unable to read camera feed")
 
# Default resolutions of the frame are obtained.The default resolutions are system dependent.
# We convert the resolutions from float to integer.
 
# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
out = cv2.VideoWriter("ouput2.avi",cv2.VideoWriter_fourcc(*'MJPG'),20.0,(224,224))
count = 0 
while(True):
    ret, frame = cap.read()
    frame = cv2.resize(frame,(224,224))
    X = preprocess_img(frame)
    vgg = vgg_avg_pooling(shape=shape)
    content_model = Model(vgg.input,vgg.layers[13].get_output_at(0))
    content_target = content_model.predict(X)
    symb_conv_outputs = [layer.get_output_at(0) for layer in vgg.layers if layer.name.endswith('conv1')]
    multi_output_model = Model(vgg.input, symb_conv_outputs)
    symb_layer_out = [K.variable(y) for y in multi_output_model.predict(style_img)]
    weights = [0.2,0.4,0.3,0.5,0.2]
    
    loss=K.mean(K.square(content_model.output-content_target)) *4 
    for symb,actual,w in zip(symb_conv_outputs,symb_layer_out,weights):
        loss += 0.03 * w * style_loss(symb[0],actual[0])
    
    grad = K.gradients(loss,vgg.input)
    get_loss_grad = K.function(inputs=[vgg.input], outputs=[loss] + grad)
    def get_loss_grad_wrapper(x_vec):
        l,g = get_loss_grad([x_vec.reshape(*batch_shape)])
        return l.astype(np.float64), g.flatten().astype(np.float64)
    final_img = min_loss(fn=get_loss_grad_wrapper,epochs=30,batch_shape=batch_shape)
    plt.imshow(scale(final_img))
    plt.show()
    
    
    #cv2.imshow('output',frame)
    #out.write(final_img)
    filename ="frame%d.jpg" % count;count+=1 
    cv2.imwrite(filename, final_img)
    #if cv2.waitKey(1) & 0xFF == ord('q'):
        #break
     
    # Write the frame into the file 'output.avi'
        
 
    # Display the resulting frame    
        
  # Break the loop
    #if k == 27:
    #    break 
 
# When everything done, release the video capture and video write objects
cap.release()
out.release()
cv2.destroyAllWindows()

#### The last step is to recombine/stitch the generated image frames to form stylized video. The video generated with some pause in between.There are a few online free converters(online-convert,Wondershare UniConverter etc.),which can make the video smoother.

In [0]:
#Converting images to video clipping

image_folder = "./"
video_name = "ouput2.avi"

images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 1, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

In [0]:
from google.colab import files
files.download('ouput2.avi') 

## Further Improvement and Experiments:

### Following are a few strategies to improve the quality of the generated image:

#### 1) More Iterations: It is is more obvious that running the network for more iterations(approximately 1000)will reduce the overall loss and will create more clear blended images.

#### 2) Advanced CNN architecture: More often than not for NST application deeper neural networks with very advanced connection can capture high level(spatial) and detailed texture features more accurately. So it's worth trying with other superior pretrained networks like InceptionV4, GoogLeNet, Resnet-101 etc. However runtime of these networks is significantly high for NST application with thousands of iteration and require expensive computation infrastructure like powerful GPU stacks.

#### 3) Tweaking content and style loss weights: As an experiment, I tried with 4 and 0.03 as content and style loss weights respectively, mainly to focus on capturing content as much as possible(as I am running the network for only a few iterations). However, this might not be appropriate and the ideal way to find the optimal weights is through a grid search

#### 4) Tweaking layer weights for style loss: To maximize style feature capture we need to tweak weights in respective conv layers for controlling style loss computation to optimize the extraction of textures(finer textures at early layers and higher-level features at deeper layers). Again these are hyperparameters and grid search is the ideal option.